## How many rectangles are there?
__Problem:__

You are given a number of points in 2 dimensions. Find the number of rectangles that can be formed using these points.

In [82]:
# Basic Python Libs
import numpy as np
import math
from itertools import combinations

We need at least four points to form a rectangle. We also know that all angles in a rectangle must be $90^{°}$. 

I initial thought of two different strategies to approch the problem:

1. Using vector analysis, we iterate over each point and look for pairs of vectors in a $90^{°}$ angle. If we find a pair, we check if a point exists if we translate our starting point by both vectors. If we find a new rectangle, we add it to our list.

2. Using geometric transformations, we translate one point to the origion $\left(0,0\right)$ and make a rotation so that the vectors to the points are parallel to our axes. Then we can check for a pair of four points fulfilling: ${\left(x_0, y_0\right), \left(x_0 + x_i, 0\right), \left(0, y_0 + y_i\right), \left(x_0 + x_i, y_0 + y_i\right)}$. We iterate over all points and if we find a new rectangle, we add it to our list.

The second approach will fail if we have additional points that do not form a rectangle at all. So I'll stick with my first approach.

So what we need is one function to compute the vector to go from one point to an other, one function to compute the angle between two vectors and one function to add vectors.

The angle $\phi$ between two vectors is given by: $\phi = \arccos\left(\frac{\vec{a}\cdot\vec{b}}{\Vert\vec{a}\Vert\cdot\Vert\vec{b}\Vert}\right)$

In [83]:
def vector_point_pair(p1, p2):
    return (p2[0]-p1[0], p2[1]-p1[1])

def unit_vector(vec):
    return vec / np.linalg.norm(vec)

def angle_between_vectors(vec1, vec2):
    vec1_u = unit_vector(vec1)
    vec2_u = unit_vector(vec2)
    return np.arccos(np.clip(np.dot(vec1_u, vec2_u), -1.0, 1.0))

def vector_addition(vec1, vec2):
    return (vec1[0]+vec2[0], vec1[1]+vec2[1])

In [105]:
point_list = [(0,0),
              (1,0),
              (0,1),
              (-1,0),
              (0,-1),
              (1,1),
              (-1,1),
              (1,-1),
              (-1,-1)]


# point_list = [(0,0),
#               (1,0),
#               (0,1)]

def rectangle_counter(points):
    if len(points) < 4:
        print("We need at least 4 points to form a rectangle!")
        return 0
    else:
        rec_point_list = []
        for point in point_list:
            vec_list = []
            other_points = [x for x in point_list if x != point]
            for other_point in other_points:
                vec_list.append(vector_point_pair(point, other_point))
            comb_obj = combinations(vec_list, 2)
            for comb in list(comb_obj):
                last_vec = vector_addition(comb[0], comb[1])
                if ((angle_between_vectors(comb[0], comb[1]) == math.pi/2) & (last_vec in vec_list)):
                    rec_points = [point,
                                  vector_addition(point, comb[0]),
                                  vector_addition(point, comb[1]),
                                  vector_addition(point, last_vec)]
                    #rec_list_set = list(set(rec_points))
                    #rec_list_set = list(dict.fromkeys(rec_points))
                    #if rec_list_set not in rec_point_list:
                        #print(rec_list_set)
                    #rec_point_list.append(rec_list_set)
                    rec_point_list.append(rec_points)
        return len(rec_point_list)/4

rectangle_counter(point_list)

10.0

For some reason the unique list trick using 
``
list(set(list)
``
doesn't work. I still ended up with duplicates. Hence I'll just take all permutations and divide the result by 4.